In [ ]:

%pip install -q spytial-diagramming
from spytial import *
from spytial.annotations import *


In [ ]:
# Setup for performance metrics - create nested directories
import random
from time import sleep

# Set this to True to run performance tests
RUN_PERF = False

perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 20
SIZES = [5, 10, 25, 50]


# Singly Linked List

In [ ]:
# Conception, next in a continuous line.
@orientation(selector='next', directions=['directlyRight'])
@orientation(selector='data', directions=['below'])
class SinglyNode:
    def __init__(self, data, next_node=None):
        self.data = data
        self.next = next_node

In [ ]:
def buildSinglyLinkedList(elements):
    head = None
    for element in reversed(elements):
        head = SinglyNode(element, head)
    return head


xs = buildSinglyLinkedList([7, 3, 1, 9, 2])
diagram(xs)


## Performance

In [ ]:
if RUN_PERF:
    STRUCTURE = "singly_linked_list"
    for size in SIZES:
        elements = random.sample(range(1, 1000), size)
        sll = buildSinglyLinkedList(elements)
        
        print(f"{STRUCTURE}({size} nodes): Rendering with perf_iterations={PI}...")
        diagram(sll, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)
else:
    print("Performance testing skipped. Set RUN_PERF = True to enable.")


# Doubly Linked List

In [ ]:

# Conception, next and prev laid out in opposite directions.
@orientation(selector='next & (DoublyNode->DoublyNode)', directions=['directlyRight'])
@orientation(selector='prev & (DoublyNode->DoublyNode)', directions=['directlyLeft'])
@hideAtom(selector="NoneType + int")
@orientation(directions=["below"], selector='data & (DoublyNode->str)') #{x: DoublyNode, d : str | x.data = d}')
@attribute(field="data")
@atomColor(selector='{x : DoublyNode | x.prev in NoneType}', value='black')
class DoublyNode:
    def __init__(self, data, next_node=None, prev_node=None):
        self.data = data
        self.next = next_node
        self.prev = prev_node



![dll](img/doubly-linked-list.png)

In [ ]:
def buildDoublyLinkedList(elements):
    head = None
    prev_node = None
    for element in elements:
        new_node = DoublyNode(element, None, prev_node)
        if prev_node is not None:
            prev_node.next = new_node
        else:
            head = new_node
        prev_node = new_node
    return head

ys = buildDoublyLinkedList([25, 9, 16, 4, 1])
diagram(ys)


## Performance

In [ ]:
if RUN_PERF:
    STRUCTURE = "doubly_linked_list"
    for size in SIZES:
        elements = random.sample(range(1, 1000), size)
        dll = buildDoublyLinkedList(elements)
        
        print(f"{STRUCTURE}({size} nodes): Rendering with perf_iterations={PI}...")
        diagram(dll, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)
else:
    print("Performance testing skipped. Set RUN_PERF = True to enable.")


# Circular Doubly Linked List with Sentinel

Set `VIEW` to `CYCLIC` to see thelinked list laid out cyclically, or `LINEAR` to see it similar to CLRS Fig 10.4

In [ ]:

VIEW = "CYCLIC"  # Change to "LINEAR" for linear layout,as in CLRS Figure 10.4

@attribute(selector='CircularNode', field='data')
@atomColor(selector='{x : CircularNode | x.data in NoneType}', value='black')
@hideAtom(selector="NoneType + int")
@apply_if(
    VIEW=="CYCLIC",
cyclic(selector='next & (CircularNode->CircularNode)', direction='clockwise')
)
@apply_if(
    VIEW=="LINEAR",
orientation(selector='next & (CircularNode -> (CircularNode - NoneType.~data))', directions=['directlyRight'])
)
class CircularNode:
    def __init__(self, data, next_node=None, prev_node=None):
        self.data = data
        self.next = next_node
        self.prev = prev_node


def buildCircularDoublyLinkedList(elements):
    """
    Build a circular doubly linked list with a sentinel node.
    The sentinel acts as both head and tail (prev of first == sentinel, next of last == sentinel).
    """
    sentinel = CircularNode(data=None)
    sentinel.next = sentinel
    sentinel.prev = sentinel
    
    current = sentinel
    for element in elements:
        new_node = CircularNode(data=element)
        # Insert after current
        new_node.next = current.next
        new_node.prev = current
        current.next.prev = new_node
        current.next = new_node
        current = new_node
    
    return sentinel





![cdd](img/circular-doubly-linked-list.png)

In [ ]:
# Build CLRS Figure 10.4: insert 9, 16, 4, 1 (in that order)
sentinel = buildCircularDoublyLinkedList([9, 16, 4, 1])


diagram(sentinel)


## Performance

In [ ]:
if RUN_PERF:
    STRUCTURE = "circular_doubly_linked_list"
    for size in SIZES:
        elements = random.sample(range(1, 1000), size)
        cdll = buildCircularDoublyLinkedList(elements)
        
        print(f"{STRUCTURE}({size} nodes): Rendering with perf_iterations={PI}...")
        diagram(cdll, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)
else:
    print("Performance testing skipped. Set RUN_PERF = True to enable.")
